In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [29]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [31]:
# 이미지 데이터를 로드하고 특징 벡터로 변환하는 함수
def load_and_embed_images(folder, num_clusters, target_size):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder, filename)
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image_resized = cv2.resize(image, target_size)
            image_flatten = image_resized.flatten()  # 이미지를 1차원 벡터로 변환
            images.append(image_flatten)
    images = np.array(images)
    kmeans = KMeans(n_clusters=num_clusters)
    embeddings = kmeans.fit_transform(images)
    return embeddings

In [32]:
# 불 이미지 폴더와 비불 이미지 폴더에서 특징 벡터 추출
fire_data_folder = "/Users/leekyeongho/python/fire_project/fire_img"
non_fire_data_folder = "/Users/leekyeongho/python/fire_project/non_fire_img"
num_clusters = 3  # 클러스터 개수
target_size = (128, 128)  # 이미지 크기

In [33]:
fire_embeddings = load_and_embed_images(fire_data_folder, num_clusters, target_size)
non_fire_embeddings = load_and_embed_images(non_fire_data_folder, num_clusters, target_size)

/Users/leekyeongho/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
libpng warning: iCCP: known incorrect sRGB profile
/Users/leekyeongho/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [34]:
# 불 이미지와 비불 이미지에 대한 라벨 생성
num_fire_images = len(fire_embeddings)
num_non_fire_images = len(non_fire_embeddings)
labels_fire = np.ones(num_fire_images)
labels_non_fire = np.zeros(num_non_fire_images)
labels = np.concatenate((labels_fire, labels_non_fire))

In [35]:
# 불 이미지와 비불 이미지를 합친 후 섞기
all_embeddings = np.concatenate((fire_embeddings, non_fire_embeddings))
all_labels = np.concatenate((labels_fire, labels_non_fire))
shuffle_indices = np.arange(len(all_embeddings))
np.random.shuffle(shuffle_indices)
all_embeddings = all_embeddings[shuffle_indices]
all_labels = all_labels[shuffle_indices]

In [36]:
# 랜덤 포레스트 분류기 학습
clf = RandomForestClassifier(random_state=0)
clf.fit(all_embeddings, all_labels)

RandomForestClassifier(random_state=0)

In [38]:
# 새로운 이미지 폴더에서 분류 및 예측
new_image_folder = "/Users/leekyeongho/python/fire_project/new_fire_img"
new_embeddings = load_and_embed_images(new_image_folder, num_clusters, target_size)
new_predictions = clf.predict_proba(new_embeddings)[:, 1] * 100

/Users/leekyeongho/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [39]:
# 결과 출력
for i, prediction in enumerate(new_predictions):
    print(f"Image {i+1}: 확률 {prediction:.2f}%")

Image 1: 확률 99.00%
Image 2: 확률 100.00%
Image 3: 확률 53.00%
Image 4: 확률 58.00%
Image 5: 확률 100.00%
Image 6: 확률 100.00%
Image 7: 확률 94.00%
Image 8: 확률 57.00%
Image 9: 확률 100.00%
Image 10: 확률 50.00%
Image 11: 확률 93.00%
Image 12: 확률 100.00%
Image 13: 확률 100.00%
Image 14: 확률 100.00%
Image 15: 확률 99.00%
Image 16: 확률 52.00%
Image 17: 확률 43.00%
